In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/vehicle-dataset-from-cardekho/Car details v3.csv')
df

## Data Cleaning

### Part 1. Dealing with numerical NaN's

In [ ]:
df = df.drop('name', axis=1)

In [ ]:
df

In [ ]:
df.isna().sum()

#### Replacing null values and doing additional manipulation for 'mileage' column

In [ ]:
# for rindex, rrindex in enumerate(df['mileage']):
#     df['mileage'] = df['mileage'][rindex].replace(' kmpl', '')
#     print(rindex)

df['mileage'].replace(' kmpl', '', inplace=True, regex=True)
df['mileage'].replace(' km/kg', '', inplace=True, regex=True)


# df['mileage'].apply(lambda x: x.replace(' kmpl', ''))
#     f[1] = df[1].apply(lambda x: x + 1)
# pd.to_numeric(df['mileage'], errors='raise')
# df

In [ ]:
pd.to_numeric(df['mileage'])

In [ ]:
df_2 = df[~df['mileage'].isnull()]
#df[df['column name'].isnull()]
mileages = pd.to_numeric(df_2['mileage'])
impute_value = mileages.mean()
print(f'Value for imputation (based off the mean of mileages): {round(impute_value, 2)}')

In [ ]:
df.mileage = df.mileage.fillna(value=round(impute_value, 2))
df.mileage.isna().sum()
df.mileage = pd.to_numeric(df.mileage)

In [ ]:
df

#### 'Engine' column

In [ ]:
df.engine.isna().sum()

In [ ]:
# Same process as 'mileage' column
# df['mileage'].replace(' kmpl', '', inplace=True, regex=True)
df.engine.replace(' CC', '', inplace=True, regex=True)
pd.to_numeric(df.engine)
df_engines = df[~df['engine'].isnull()]
#df[df['column name'].isnull()]
engines = pd.to_numeric(df_engines['engine'])
impute_value_engine = int(round(engines.mean(), 0))
print(f'Value for imputation (based off the mean of mileages): {impute_value_engine}')

#Perform imputation
df.engine = df.engine.fillna(value=impute_value_engine)
df.engine = pd.to_numeric(df.engine)

#Check results
print(f'Number of nans: {df.engine.isna().sum()}')

#### Repeating the process for 'max_power'

In [ ]:
#Same process as above (I should really have a function built for this by now!)
# Same process as 'mileage' column
# df['mileage'].replace(' kmpl', '', inplace=True, regex=True)
df.max_power.replace(' bhp', '', inplace=True, regex=True)
pd.to_numeric(df.max_power)
df_mp = df[~df['max_power'].isnull()]
#df[df['column name'].isnull()]
mp = pd.to_numeric(df_mp['max_power'])
impute_value_mp = int(round(mp.mean(), 1))
print(f'Value for imputation (based off the mean of mileages): {impute_value_mp}')

#Perform imputation
df.max_power = df.max_power.fillna(value=impute_value_mp)
df.max_power = pd.to_numeric(df.max_power)

#Check results
print(f'Number of nans: {df.max_power.isna().sum()}')

#### Dropping the 'torque' column

In [ ]:
df_nt = df.drop('torque', axis=1)

In [ ]:
df_nt

In [ ]:
df_nt.dtypes

#### Dealing with 'seats'

In [ ]:
df_nt.seats.value_counts()

In [ ]:
#We'll be replacing missing values for the 'seats' column with '5.0', the mode.

In [ ]:
df_nt.seats = df_nt.seats.fillna(value=5.0)
print(f'Number of nans for seats column: {df_nt.seats.isna().sum()}')

### Part 2. Dealing with categorical values

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Basic plotting to get a better understanding of categorical columns
cat_cols = ['fuel', 'seller_type', 'transmission', 'owner']

for i in cat_cols:
    sns.countplot(x=i, data = df_nt)
    plt.show()

In [ ]:
# We can use OHE and OrdinalEncoding to deal with all categorical columns swiftly and efficiently
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

#### Dealing with 'Transmission' column

In [ ]:
#Simple binary encoding, 0 if manual, 1 if automatic
mapper = {'Manual':0, 'Automatic':1}
df_nt.transmission = df_nt.transmission.replace(mapper)
df_nt.transmission.value_counts()

#### OHE

In [ ]:
import numpy as np

In [ ]:
# cols = ['fuel', 'seller_type']
# ohe = OneHotEncoder()
# feature_array = ohe.fit_transform(df[['fuel', 'seller_type']]).toarray()
# feature_array

In [ ]:
# feature_labels = ohe.categories_
# # feature_labels =  np.ndarray.flatten(feature_labels)
# feature_labels

In [ ]:
# cols = ['CNG', 'Diesel', 'LPG', 'Petrol', 'Dealer', 'Individual' ,'Trustmark Dealer']
# # '1983', '1991', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
#         #'2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
#        # '2014', '2015', '2016', '2017', '2018', '2019', '2020'

In [ ]:
# df_OHE = pd.DataFrame(feature_array)

# df_OHE
# # feature_array

In [ ]:
# df_nt = df_nt.drop(['fuel', 'seller_type'], axis=1)

In [ ]:
# df_nt

In [ ]:
# df_nt = pd.concat([df_nt, df_OHE], axis=1)

In [ ]:
# df_nt

#### Dealing with 'owner' column and OrdinalEncoder

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
df.fuel.unique()
# df.seller_type.unique()

In [ ]:
df_nt

In [ ]:
#Wrapper for seller_type
mapp = {'Individual':0, 'Dealer':1, 'Trustmark Dealer':2}
df_nt.seller_type = df_nt.seller_type.replace(mapp)

#Wrapper for fuel
mapp2 = {'Diesel': 0, 'Petrol': 1, 'LPG': 2, 'CNG': 3}
df_nt.fuel= df_nt.fuel.replace(mapp2)

#Simple binary encoding, 0 if manual, 1 if automatic
mapper = {'Manual':0, 'Automatic':1}
df_nt.transmission = df_nt.transmission.replace(mapper)
df_nt.transmission.value_counts()

In [ ]:
df_nt

In [ ]:
# model_ODE = ColumnTransformer(
#     [('ODE', OrdinalEncoder(),['owner'])],
#     remainder = 'passthrough'
#     )

# fuix = model_ODE.fit_transform(df_nt)
# pd.DataFrame(fuix) 
# df_nt.owner.unique()
cats = ['First Owner', 'Second Owner', 'Third Owner', 'Fourth & Above Owner', 'Test Drive Car']
fuel_cats = []
seller_type_cats = []

# cats
ordi = OrdinalEncoder(categories=[cats])
ordi.fit(df_nt[['owner']])

In [ ]:
df_nt.owner = pd.DataFrame(ordi.transform(df_nt[['owner']])) 

Additional fixes

In [ ]:
df_nt.max_power = df_nt.max_power.fillna(impute_value_mp)
df_nt.max_power.isna().sum()

df_nt.isna().sum().sum()

#### Dealing with 'year'

### Model Splitting / Training

In [ ]:
df_nt.dtypes

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

In [ ]:
y = df_nt.selling_price
X = df_nt.drop('selling_price', axis=1)
# X = df_nt.drop(['selling_price', 'km_driven'], axis=1)

In [ ]:
#Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = 1)

X_train

In [ ]:
#Scaling the data for Linear Regression
    
scaler = StandardScaler()

scaled_X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
scaled_X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

# Running linear regression

LR = LinearRegression()
LR.fit(scaled_X_train, y_train)
y_pred_LR = LR.predict(scaled_X_test)

X_train
scaled_X_train


Assessing the model

In [ ]:
xgbr = XGBRegressor(verbosity=0)
xgbr.fit(X_train, y_train)
# print(xgbr.score(scaled_X_train, y_train))

y_pred_X = xgbr.predict(X_test)

### Assessing model performance

In [ ]:
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse
import math

In [ ]:
def get_metrics(preds, actual, model_type):
    r2s = r2(preds, actual)
    maes = mae(preds, actual)
    mses = mse(preds, actual)
    rmses = math.sqrt(mses)
#     return rmse, mse, mae, r2
    df = pd.DataFrame([r2s, maes, mses, rmses, model_type]).transpose()
    df.columns = ['r2 coeff.', 'MAE', 'MSE', 'RMSE', 'Type of model']
    return df

In [ ]:
df_metrics = get_metrics(y_pred_LR, y_test, 'Linear Regression')
df_metrics_X = get_metrics(y_pred_X, y_test, 'XGBR')

df_metrics = pd.concat([df_metrics, df_metrics_X], axis=0)
df_metrics

# Back to the drawing-board
### Let's perform some feature engineering and attempt to fix-up our model

In [ ]:
from matplotlib import rcParams

In [ ]:
# sns.pairplot(data=df)

In [ ]:
rcParams['figure.figsize'] = 15,8
sns.histplot(x='selling_price', hue='owner', data=df, log_scale=True);

We can see that it's pretty heavily skewed, so if you're a first owner that's a lot better than the second third fourth and test drive cars

In [ ]:
df.owner.value_counts()/ len(df.owner)

For this, let's combine the classes of '2nd', '3rd', 'fourth and above', and 'test drive car into one variable'

In [ ]:
#Wrapper
wr = {'First Owner': 0, 'Second Owner': 1, 'Third Owner': 1, 'Fourth & Above Owner': 1, 'Test Drive Car': 1}
df.owner = df.owner.replace(wr)
df.owner.value_counts()

In [ ]:
sns.histplot(x='selling_price', hue='owner', data=df, log_scale=True);

In [ ]:
df.fuel.dtype

In [ ]:
numerical_cols = ['selling_price', 'km_driven', 'year', 'engine', 'mileage', 'max_power', 'seats']
for col in numerical_cols:
    df.boxplot(col)
    plt.show()

Seem to be some outliers, especially that one on the top. Let's get rid of that value using z-scores

In [ ]:
from scipy.stats import zscore

In [ ]:
def gen_zscores(df, col):
    df['zscore_'+col] = zscore(df[col])

[gen_zscores(df, nc) for nc in numerical_cols]

df

In [ ]:
# df = df.drop('zscore_max_power', axis=1)
# df = df.drop('zscore_seats', axis=1)

In [ ]:
df.describe()

In [ ]:
df[df['zscore_selling_price'] > 5]

In [ ]:
df = df[(df.zscore_selling_price>-5) & (df.zscore_selling_price<5)]

In [ ]:
df = df.drop('zscore_selling_price', axis=1)

In [ ]:
def rid_outliers(df, col, zscore_val):
    df = df.copy()
    df = df[(df[col]>-(zscore_val)) & (df[col]<zscore_val)]
    df = df.drop([col], axis=1)
    return df

In [ ]:
df = rid_outliers(df, 'zscore_km_driven', 4.5)
# df = df.drop('zscore_km_driven', axis=1)

df = df.drop([8042, 316], axis=0)

df

In [ ]:
df = df.drop('zscore_year', axis=1)

In [ ]:
# df[df.zscore_mileage > 3.0]
df = rid_outliers(df, 'zscore_mileage', 3.0)

In [ ]:
df = df.drop(['zscore_max_power', 'zscore_seats'], axis=1)
df

In [ ]:
df = rid_outliers(df, 'zscore_engine', 3.4)

In [ ]:
numerical_cols = ['selling_price', 'km_driven', 'year', 'engine', 'mileage', 'max_power', 'seats']
for col in numerical_cols:
    df.boxplot(col)
    plt.show()

Data looks a bit better in terms of outliers, let's add one final touch

In [ ]:
df[df.seats == 14.0]

In [ ]:
#From above, the index is 4575, dropping this index due to the seat count being so high (probably a bus, we're trying to predict car selling prices)
df = df.drop(4575)
df

In [ ]:
[gen_zscores(df, nc) for nc in numerical_cols]
df

In [ ]:
zs = ['zscore_selling_price', 'zscore_km_driven', 'zscore_year', 'zscore_engine', 'zscore_mileage']

for z in zs:
    sns.boxplot(df[z])
    plt.show()

#### Let's leave the outlier trimming at this for the time being, we can always return to it and cut some additional data out if our model is not performing to expectations

In [ ]:
df = df.drop(zs, axis=1)

In [ ]:
df = df.drop(['zscore_max_power', 'zscore_seats'], axis=1)

### Additional EDA

In [ ]:
cats = ['fuel', 'seller_type', 'transmission']
for c in cats:
    sns.countplot(x=c, data=df)
    plt.show()

As we previously have done, we'll simply be label encoding the categorical columns in the following manner:
1. For the seller type, the individual can be a '0', the dealer will be '1', and the trustmark dealer will be '2'.
2. For fuel we're going to modify it slightly from our previous run: Diesel will be 0, petrol will be 1, and 'LPG' and 'CNG' will both become 2 (the two will effectively be combined into one).
3. Simple binary encoding for transmission, 0 if manual, 1 if automatic

In [ ]:
#Wrapper for seller_type
mapp = {'Individual':0, 'Dealer':1, 'Trustmark Dealer':2}
df.seller_type = df.seller_type.replace(mapp)

#Wrapper for fuel
mapp2 = {'Diesel': 0, 'Petrol': 1, 'LPG': 2, 'CNG': 2}
df.fuel= df.fuel.replace(mapp2)

#Simple binary encoding, 0 if manual, 1 if automatic
mapper = {'Manual':0, 'Automatic':1}
df.transmission = df.transmission.replace(mapper)
df.transmission.value_counts()

df

In [ ]:
df.isna().sum()

In [ ]:
sns.histplot(x='seats', data=df)

In [ ]:
# We will impute the seats NaN values with 5, the most common occurance
df.seats = df.seats.fillna(value=5.0)
print(f'Number of nans for seats column: {df.seats.isna().sum()}')

In [ ]:
df = df.drop(['torque'], axis = 1)
df

In [ ]:
df.max_power = df.max_power.fillna(value=impute_value_mp)
df.isna().sum()

## Scaling and building our model

In [ ]:
y = df.selling_price
X = df.drop('selling_price', axis=1)

### Linear Regression

In [ ]:
#Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 1)

#Scaling the data for Linear Regression
    
scaler = StandardScaler()

scaled_X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
scaled_X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

# Running linear regression

LR_2 = LinearRegression()
LR_2.fit(scaled_X_train, y_train)
y_pred_LR_2 = LR_2.predict(scaled_X_test)


### XGBoostRegressor

In [ ]:
xgbr_2 = XGBRegressor(verbosity=0)
xgbr_2.fit(X_train, y_train)
y_pred_X_2 = xgbr_2.predict(X_test)

In [ ]:
df_metrics

In [ ]:
df_metrics_LR2 = get_metrics(y_pred_LR_2, y_test, 'Linear Regression - Revised')
df_metrics_X2 = get_metrics(y_pred_X_2, y_test, 'XGBR - revised')

df_metrics = pd.concat([df_metrics, df_metrics_LR2, df_metrics_X2], axis=0)
df_metrics

Conclusion: Linear Regression improved ever so slightly, XGBR got worst

## Hyperparameter Optimization

In [ ]:
#To be continued...